In [37]:
#IMPORTS 

import neurolThesis
from neurolThesis import streams
from neurolThesis.connect_device import get_lsl_EEG_inlets
from neurolThesis.BCI import generic_BCI, automl_BCI
from neurolThesis import BCI_tools
from neurolThesis.models import classification_tools
from sys import exit
from pylsl import StreamInlet, resolve_stream
import numpy as np
import threading
import wave
import time
import tkinter as tk
from tkinter import ttk  # For the ComboBox
import random
import pyaudio

In [38]:
#AUDIO FILES + FUNCTIONS

'''Audio Files'''
# dictionary of audio files corresponding to each box
audio_files = {
    "C7": r"Notes/notes_A.wav",
    "B": r"Notes/notes_B.wav",
    "A": r"Notes/notes_C.wav",
    "D": r"Notes/notes_D.wav",
    "E": r"Notes/notes_E.wav",
    "F": r"Notes/notes_F.wav",
    "G": r"Notes/notes_G.wav",
    "C8": r"Notes/notes_Gs.wav"
}


audio_files_by_key = {
    "C": {
        "C": r"Notes/C5.wav",
        "B": r"Notes/B6.wav",
        "A": r"Notes/A6.wav",
        "D": r"Notes/D5.wav",
        "E": r"Notes/E5.wav",
        "F": r"Notes/F5.wav",
        "G": r"Notes/G5.wav",
        "C8": r"Notes/C6.wav"
    },
    "D": {
        "D": r"Notes/notes_A.wav",
        "E": r"Notes/notes_B.wav",
        "F#": r"Notes/notes_C.wav",
        "G": r"Notes/notes_D.wav",
        "A": r"Notes/notes_E.wav",
        "B": r"Notes/notes_F.wav",
        "C#": r"Notes/notes_G.wav",
        "D8": r"Notes/notes_Gs.wav"
    },
    "G": {
        "G": r"Notes/notes_A.wav",
        "A": r"Notes/notes_B.wav",
        "B": r"Notes/notes_C.wav",
        "C": r"Notes/notes_D.wav",
        "D": r"Notes/notes_E.wav",
        "E": r"Notes/notes_F.wav",
        "F#": r"Notes/notes_G.wav",
        "G8": r"Notes/notes_Gs.wav"
    },
    "A": {
        "A": r"Notes/notes_A.wav",
        "B": r"Notes/notes_B.wav",
        "C#": r"Notes/notes_C.wav",
        "D": r"Notes/notes_D.wav",
        "E": r"Notes/notes_E.wav",
        "F#": r"Notes/notes_F.wav",
        "G#": r"Notes/notes_G.wav",
        "A8": r"Notes/notes_Gs.wav"
    },
    "Chords": {
        "C": r"Notes/CChord.wav",
        "G": r"Notes/GChord.wav",
        "Am": r"Notes/AmChord.wav",
        "F": r"Notes/FChord.wav",
        "E": r"Notes/notes_E.wav",
        "F#": r"Notes/notes_F.wav",
        "G#": r"Notes/notes_G.wav",
        "A8": r"Notes/notes_Gs.wav"
    }
}


# Global variables to manage audio playback thread
stop_thread = False
audio_thread = None


'''Play Audio Function'''

def play_audio(file_path):
    global audio_thread

    # Define a function to handle audio playback
    def play_thread(path):
        # Open the WAV file
        wf = wave.open(path)

        # Create a PyAudio instance
        p = pyaudio.PyAudio()

        # Open a stream
        stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                        channels=wf.getnchannels(),
                        rate=wf.getframerate(),
                        output=True)

        # Read data in chunks
        data = wf.readframes(1024)

        # Play stream (3)
        while len(data) > 0:
            stream.write(data)
            data = wf.readframes(1024)

        # Stop stream
        stream.stop_stream()
        stream.close()

        # Close PyAudio
        p.terminate()

    # Stop any currently playing audio
    stop_audio()
    play_thread(file_path)
    # Start a new thread for playback
    #audio_thread = threading.Thread(target=play_thread, args=(file_path,))
    #audio_thread.start()


'''Stop Audio and Join Threads Function'''
def stop_audio():
    global audio_thread
    # If there's an audio thread running, wait for it to finish
    if audio_thread is not None:
        audio_thread.join()
    # Reset the thread variable
    audio_thread = None


In [39]:
#TKINTER FUNCTIONS

'''Generates a letter from the selected key'''
def generate_letter(note):
    global root, selected_key
    #note = random.randint(0, 7)
    print('Note generated', note)
    if selected_key == 'Chords':
        '''
        if note == 2 or note == 3:
            note = 1
        elif note == 4 or note == 5 or note == 6:
            note = 2
        elif note == 7 or note == 8:
            note = 3
        '''
        if note == 2:
            note = 1
        elif note == 3:
            note = 2
        elif note == 4 or note == 5 or note == 6 or note == 7 or note == 8:
            note = 3
    #print('Note generated', note)
    if all(position == 'normal' for position in box_positions.values()):
        letters = list(boxes.keys())
        selected_letter = letters[note]
        raise_box(selected_letter)
    #root.after(2000, generate_letter)

'''Highlights the selected letter'''
def raise_box(letter):
    global current_highlighted, selected_key
    
    # Reset all boxes to their original state
    for box in boxes.values():
        box.config(highlightbackground="white", highlightthickness=1)
        box.grid_configure(pady=0)  # Reset any padding changes
    
    # Highlight the selected box
    selected_box = boxes[letter]
    selected_box.config(highlightbackground="black", highlightthickness=10)
    current_highlighted = letter

'''Moves the selected letter down and plays the note'''
def move_box(selected_key):
    global current_speaker
    global current_highlighted
    if current_highlighted:
        # Reset padding for all boxes
        for letter, box in boxes.items():
            box.grid_configure(pady=(0, 20))

        # Adjust padding for the highlighted box
        box = boxes[current_highlighted]
        if box_positions[current_highlighted] == 'normal':
            box.grid_configure(pady=(20, 0))  # Move the highlighted box up
            box_positions[current_highlighted] = 'raised'
            audio_path = audio_files_by_key[selected_key][current_highlighted]
            threading.Thread(target=play_audio, args=(audio_path,)).start()
        else:
            box.grid_configure(pady=(0, 20))  # Move the highlighted box down
            box_positions[current_highlighted] = 'normal'
            stop_audio()
    
    #time.sleep(300)
    #box.grid_configure(pady=(0, 20))  # Move the highlighted box down
    #box_positions[current_highlighted] = 'normal'
    #stop_audio()
    #root.unbind('<space>')
    #root.after(500,move_box(selected_key))
    

'''Ends Application'''
def exit_application():
    global root
    """Function to exit the application."""
    root.quit()
    root.destroy()  # Close the Tkinter window
    # Optionally, add any clean-up code here
    exit()  # Terminate the Python program



'''Starts the calibrator'''
def start_bci_and_calibration():
    global root, start_button, selected_key
    """Function to start BCI and calibration and disable the start button."""
    # Disable the start button to prevent further clicks
    # start_button.config(state=tk.DISABLED)
    root.unbind('c')
    # Start the BCI and calibration in a new thread
    #generate_letter()
    threading.Thread(target=run_bci, daemon=True).start()

In [40]:
#CLASSIFIERS
'''Classifiers for a single band, two bands, three bands, and a single electrode'''

def clfSingle(clf_input, clb_info):
    # Convert inputs to numpy arrays
    clf_input_np = np.array(clf_input[:8])  # Only consider the first 8 elements
    clb_info_np = np.array(clb_info)[:, :7]  # Consider only the 7 percentiles
    #print('clf_input_np:', clf_input_np)
    #print('\n clb_info_np:',clb_info_np)
    # Compare clf_input_np with each percentile in clb_info_np and count how many percentiles it exceeds
    bin_indices = np.sum(clf_input_np[:, np.newaxis] > clb_info_np, axis=0) 
    #print('\nclf_input_np[:, np.newaxis] :', clf_input_np[:, np.newaxis] )
    #print('\nclb_info_np', clb_info_np)

    
    note = round(np.average(bin_indices))
    
    return note


def clfDouble(clf_input, clb_info):
    
    #convert into numpy arrays
    clf1 = np.array(clf_input[0][:8])
    clf2 = np.array(clf_input[1][:8])
    print('classifier input1', clf1)
    
    clb1 = np.array(clb_info)[:,0,:8]
    clb2 = np.array(clb_info)[:,1,:8]
    print('calibration thresholds1', clb1)

    print('classifier input2', clf2)
    print('calibration thresholds2', clb2)

    bin_indices1 = np.sum(clf1[:, np.newaxis] > clb1, axis=0) 
    bin_indices2 = np.sum(clf2[:, np.newaxis] > clb2, axis=0)
    
    note = round((np.average(bin_indices1) + np.average(bin_indices2))/2)
    
    return note


def clfTriple(clf_input, clb_info):
    
    #convert into numpy arrays
    clf1 = np.array(clf_input[0][:8])
    clf2 = np.array(clf_input[1][:8])
    clf3 = np.array(clf_input[2][:8])
    
    clb1 = np.array(clb_info)[:,0,:7]
    clb2 = np.array(clb_info)[:,1,:7]
    clb3 = np.array(clb_info)[:,2,:7]
    
    bin_indices1 = np.sum(clf1[:, np.newaxis] > clb1, axis=0) 
    bin_indices2 = np.sum(clf2[:, np.newaxis] > clb2, axis=0)
    bin_indices3 = np.sum(clf3[:, np.newaxis] > clb3, axis=0)
    
    note = round((np.average(bin_indices1) + np.average(bin_indices2) + np.average(bin_indices3) )/3)
    
    return note


def clfElectrode(clf_input, clb_info, electrode):
    
    n = electrode - 1
    # Convert inputs to numpy arrays
    clf_input_np = np.array(clf_input[n])  # Only consider the chosen electrode 
    clb_info_np = np.array(clb_info)[:7, n]  # Consider only the 7 percentiles
    
    print('clf_input_np:', clf_input_np)
    print('\n clb_info_np:',clb_info_np)
    

    bin_indices = np.sum(clf_input_np[np.newaxis] > clb_info_np, axis=0) 

    note = round(np.average(bin_indices))
    
    return note

In [41]:
#MAIN ROOT

# Box dimensions
width = 150
height = 100

colors = ["red", "orange","green", "SeaGreen3", "cyan", "blue",  "purple", "pink"]
# Dictionary mapping musical keys to arrays of letters
key_to_letters = {
    "C": ["C", "D", "E", "F", "G", "A", "B", "C8"],
    "D": ["D", "E", "F#", "G", "A", "B", "C#", "D8"],
    "G": ["G", "A", "B", "C", "D", "E", "F#", "G8"],
    "A": ["A", "B", "C#", "D", "E", "F#", "G#", "A8"],
    "Chords": ["C", "G", "Am", "F"]
}

def main_app(selected_key):
    global root, boxes, box_positions, start_button
    root = tk.Tk()
    root.title("EEGstrument")
    
    # Default letters array if the selected key is not found
    default_letters = ["A7", "B", "C", "D", "E", "F", "G", "A8"]
    letters = key_to_letters.get(selected_key, default_letters)
    
    # Create and place the colored boxes with labels
    boxes = {}
    box_positions = {}  # Track the position of each box
    
    # Adjust window size
    window_width = width * len(letters)
    window_height = height + 150  # Extra space for buttons
    root.geometry(f"{window_width}x{window_height}")
    
    # Informative label at the top of the window
    tk.Label(root, text="For Calibration: Press 'C' then open and close your left hand every second for 20 seconds. \n\nPress Space to Play Selected Note!").grid(row=0, column=0, columnspan=len(colors))
    
    # Create and place colored boxes with labels
    for i, (color, letter) in enumerate(zip(colors, letters)):
        frame = tk.Frame(root, bg=color, width=width, height=height, highlightbackground="white", highlightthickness=1)
        frame.grid(row=1, column=i, pady=(20, 20))  # Adjusted padding for visual separation
        frame.pack_propagate(False)
        label = tk.Label(frame, text=letter, bg=color, font=("Helvetica",16,"bold"))
        label.pack(expand=True)  # Fill the frame
        boxes[letter] = frame
        box_positions[letter] = 'normal'
    
    # Create a frame for buttons
    button_frame = tk.Frame(root)
    button_frame.grid(row=2, column=0, columnspan=len(colors), pady=30)
    
    # Button configuration within the button_frame using grid
    #move_button = tk.Button(button_frame, text="Move Box", command=move_box)
    #move_button.grid(row=0, column=0, padx=10)
    # Bind the spacebar key to the move_box function
    root.bind('<space>', lambda event: move_box(selected_key))
    root.bind('c', lambda event: start_bci_and_calibration())

    # start_button = tk.Button(button_frame, text="Start Calibration", command=start_bci_and_calibration)
    # start_button.grid(row=0, column=1, padx=10)
    exit_button = tk.Button(button_frame, text="Exit", command=exit_application)
    exit_button.grid(row=0, column=2, padx=10)
    
    # Start the main event loop
    root.mainloop()

In [42]:
#SPLASH ROOT

def on_start():
    global selected_key
    selected_key = key_dropdown.get()  # Get the selected musical key from the dropdown
    print(f"Selected Musical Key: {selected_key}")
    
    # Destroy the splash screen
    splash_root.destroy()
    
    # Initialize the main application window
    main_app(selected_key)

# Create the splash screen window
splash_root = tk.Tk()
splash_root.title("Splash Screen")
splash_root.geometry("300x150")

musical_keys = ["C", "D", "G", "A", "Chords"]
key_dropdown = ttk.Combobox(splash_root, values=musical_keys, state="readonly")
key_dropdown.pack(pady=10)
key_dropdown.set("Choose a Key")  # Default/placeholder text

# Add a 'Start' button to the splash screen
start_button_splash = tk.Button(splash_root, text="Start", command=on_start)
start_button_splash.pack(pady=20)

In [43]:
#LSL STREAM

streams1 = resolve_stream("name='Unicorn'")
inlet = StreamInlet(streams1[0])
stream = streams.lsl_stream(inlet, buffer_length=250)

In [44]:
#BCI INITIALIZATION
clb = lambda stream:  BCI_tools.band_power_calibrator(stream, ['EEG 1', 'EEG 2', 'EEG 3', 'EEG 4', 
                                                               'EEG 5', 'EEG 6', 'EEG 7', 'EEG 8'], 'unicorn', 
                                                        bands=['alpha_low','alpha_high'],
                                                        percentile=50, recording_length=10, epoch_len=1, inter_window_interval=1)


gen_tfrm = lambda signal, clb_info: BCI_tools.band_power_transformer(signal,sampling_rate=250,bands=['alpha_low','alpha_high'])#, ['EEG 1', 'EEG 2', 'EEG 3', 'EEG 4', 
                                                               #'EEG 5', 'EEG 6', 'EEG 7', 'EEG 8'],'unicorn', bands=['alpha_low','alpha_high'], epoch_len=1)
BCI = generic_BCI(clfDouble, transformer=gen_tfrm, action=generate_letter, calibrator=clb)

def run_bci():
    BCI.calibrate(stream)
    BCI.run(stream)


# Create a thread for BCI operations
bci_thread = threading.Thread(target=run_bci, daemon=True)


In [45]:
#RUN PROGRAM

# Start the event loop for the splash screen
splash_root.mainloop()

Selected Musical Key: Chords
Recording for 10 seconds...

Computed the following power percentiles: 
[[[4.78426529e+02 5.00376641e+02 5.16569105e+02 5.30726197e+02
   5.09900572e+02 5.69204758e+02 5.78513695e+02 5.92463689e+02
   1.92697228e-07 3.94447394e-08 5.88439509e-09 5.43339167e-04
   1.04087665e-03 1.78703408e-03 4.07093577e-05 5.41905825e+02
   6.36083714e-09]
  [1.02984626e+02 1.08903563e+02 1.11696918e+02 1.15357742e+02
   1.13297028e+02 1.26314683e+02 1.28505367e+02 1.31114109e+02
   1.12651228e-07 5.87211536e-08 9.49284786e-09 3.10971821e-04
   6.78638248e-04 3.08231549e-04 8.98077530e-06 1.19548301e+02
   1.40324614e-09]]

 [[4.82572062e+02 5.03603941e+02 5.24000365e+02 5.31623227e+02
   5.18728819e+02 5.83258471e+02 5.86888778e+02 5.99788774e+02
   3.24789565e-07 9.53992115e-08 1.08453333e-08 6.04496433e-04
   1.84945430e-03 1.95417028e-03 4.07093577e-05 5.42718424e+02
   6.36083714e-09]
  [1.04263727e+02 1.10163158e+02 1.13051890e+02 1.16340777e+02
   1.13822492e+02 1.2

SystemExit: 